In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cp -av "/content/drive/MyDrive/[NN'22] Project Dataset" "/content/[NN'22] Project Dataset"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/MyDrive/[NN'\''22] Project Dataset' -> '/content/[NN'\''22] Project Dataset/[NN'\''22] Project Dataset'
cp: cannot create symbolic link '/content/[NN'\''22] Project Dataset/[NN'\''22] Project Dataset': Read-only file system


In [ ]:
import os
import random
import cv2
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Dropout,GlobalAveragePooling2D,BatchNormalization,Activation,Input,concatenate,AveragePooling2D,MaxPool2D,Input
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import to_categorical


In [ ]:
TRAIN_DIR = "/content/[NN'22] Project Dataset/Train"
IMG_SIZE = 150
INPUT_SIZE = (150,150)
MappingDict = {
    "dew":0,
    "fogsmog":1,
    "frost":2,
    "glaze":3,
    "hail":4,
    "lightning":5,
    "rain":6,
    "rainbow":7,
    "rime":8,
    "sandstorm":9,
    "snow":10

}

In [ ]:
training_data = []
training_labels = []
for cls in tqdm(os.listdir(TRAIN_DIR)):
  clspath = os.path.join(TRAIN_DIR, cls)
  for img in os.listdir(clspath):
    path = os.path.join(clspath, img)
    if 'fogsmog_495' in path:
      continue
    img_data = cv2.imread(path)
    #cv2.imshow(img_data)
    img_data = cv2.resize(img_data, INPUT_SIZE)
    img_data = img_data.astype(float) / 255
    training_data.append(img_data)
    training_labels.append(MappingDict[cls])
   # if random.randint(0,100) > 80:
   #   training_data.append(cv2.flip(img_data, 0))
   #   training_labels.append(MappingDict[cls])
label_encoder = LabelEncoder()
#training_labels = label_encoder.fit_transform(training_labels)
training_labels = to_categorical(training_labels)
    

100%|██████████| 11/11 [00:46<00:00,  4.21s/it]


In [ ]:
from keras_preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1.0/255.,
                                   rotation_range=90,
                                   width_shift_range=0.2,
                                   #height_shift_range=0.2,
                                   #shear_range=0.2,
                                   #zoom_range=0.3,
                                   validation_split=0.3,
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    batch_size=16,
                                                    shuffle=True,
                                                    class_mode='categorical',
                                                    subset='training',
                                                    target_size=INPUT_SIZE)
valid_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, 
    batch_size=16,
    shuffle=True,
    class_mode='categorical',
    subset='validation',
    target_size=INPUT_SIZE)

Found 3889 images belonging to 11 classes.
Found 1662 images belonging to 11 classes.


In [6]:
model = Sequential()
#if we are using "channels first", update the input shape
inputShape = (150, 150, 3)
#define the first (and only) CONV => RELU layer
model.add(Conv2D(256, (3, 3), input_shape=inputShape, activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(512, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(1024, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(2048, (3, 3), activation="relu"))

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(11, activation="softmax"))
#model = mod()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


x= np.array(training_data)
y = np.array(training_labels)
x, x_val, y, y_val = train_test_split(x, y, test_size=0.3, stratify=y)

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
#model.fit(train_generator, validation_data = valid_generator,epochs=100) 
model.fit(x,y ,validation_data=(x_val,y_val), epochs=100,shuffle=True,batch_size=16)
model.save("test.model")

Epoch 1/100
243/243 [==============================] - 26s 51ms/step - loss: 2.2515 - accuracy: 0.2033 - val_loss: 2.0058 - val_accuracy: 0.2775
Epoch 2/100
243/243 [==============================] - 11s 47ms/step - loss: 1.8789 - accuracy: 0.3279 - val_loss: 1.7053 - val_accuracy: 0.3808
Epoch 3/100
243/243 [==============================] - 11s 44ms/step - loss: 1.6333 - accuracy: 0.4458 - val_loss: 1.4872 - val_accuracy: 0.4865
Epoch 4/100
243/243 [==============================] - 10s 42ms/step - loss: 1.4138 - accuracy: 0.5117 - val_loss: 1.4364 - val_accuracy: 0.5099
Epoch 5/100
243/243 [==============================] - 10s 42ms/step - loss: 1.2726 - accuracy: 0.5539 - val_loss: 1.1224 - val_accuracy: 0.6030
Epoch 6/100
243/243 [==============================] - 10s 42ms/step - loss: 1.1964 - accuracy: 0.5833 - val_loss: 1.0995 - val_accuracy: 0.6192
Epoch 7/100
243/243 [==============================] - 10s 42ms/step - loss: 1.1446 - accuracy: 0.6036 - val_loss: 1.0675 - val_ac

In [7]:
print('image_name,label')
for img in os.listdir("/content/[NN'22] Project Dataset/Test"):
  if 'test_1297' in img:
    print('test_1297.jpg,10')
    continue
  imgpath = os.path.join("/content/[NN'22] Project Dataset/Test", img)
  img_data = cv2.imread(imgpath)
  img_data = cv2.resize(img_data, (150, 150))
  img_data = img_data.astype(float) / 255
  img_arr = np.array(img_data)
  data = []
  data.append(img_data)
  prediction = model.predict(np.array(data))
  classes = np.argmax(prediction, axis = 1)
  print(img +","+str(classes[0]))


image_name,label
test_366.jpg,1
test_324.jpg,1
test_338.jpg,8
test_376.jpg,1
test_388.jpg,2
test_329.jpg,6
test_452.jpg,7
test_295.jpg,2
test_418.jpg,5
test_405.jpg,1
test_385.jpg,1
test_306.jpg,1
test_293.jpg,6
test_341.jpg,2
test_437.jpg,8
test_371.jpg,2
test_464.jpg,10
test_412.jpg,2
test_299.jpg,10
test_315.jpg,8
test_302.jpg,6
test_386.jpg,3
test_312.jpg,1
test_421.jpg,9
test_344.jpg,4
test_353.jpg,1
test_463.jpg,5
test_465.jpg,6
test_461.jpg,5
test_317.jpg,2
test_438.jpg,6
test_396.jpg,3
test_448.jpg,4
test_454.jpg,0
test_422.jpg,5
test_392.jpg,2
test_417.jpg,2
test_460.jpg,0
test_433.jpg,1
test_350.jpg,8
test_340.jpg,6
test_337.jpg,2
test_291.jpg,1
test_332.jpg,5
test_416.jpg,3
test_328.jpg,5
test_348.jpg,6
test_383.jpg,8
test_345.jpg,10
test_330.jpg,5
test_435.jpg,10
test_431.jpg,8
test_318.jpg,3
test_304.jpg,5
test_309.jpg,9
test_372.jpg,3
test_462.jpg,8
test_426.jpg,9
test_356.jpg,0
test_296.jpg,1
test_305.jpg,6
test_391.jpg,8
test_406.jpg,8
test_300.jpg,2
test_427.jpg,8
test

In [8]:
model = load_model("model3")

In [ ]:
image_names=[]
labels = []
for img in os.listdir(Test_DIR):
  if 'test_1297' in img:
    image_names.append('test_1297.jpg')
    labels.append(10)
    continue
  imgpath = os.path.join(Test_DIR, img)
  img_data = cv2.imread(imgpath)
  img_data = cv2.resize(img_data, img_size)
  img_data = img_data.astype(float) / 255
  img_arr = np.array(img_data)
  data = []
  data.append(img_data)
  prediction = model.predict(np.array(data))
  classes = np.argmax(prediction, axis = 1)
  image_names.append(img)
  labels.append(str(classes[0]))
my_submission = pd.DataFrame({'image_name': image_names, 'label': labels})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)